# X-Risk Estimation

In the notebook [xrisk-database-processing]('./xrisk-database-processing.ipynb'), we processed the X-Risk estimation "database" from the EA Forum.

Here we take that data and create a risk model.

The goal is to create a risk model to estimate future risk, and tackle the question of "how long do we expect to still be around?".


## X-Risk Estimates

Before getting started, we'll go through the data from the existential risk database.

In [8]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import seaborn as srs
import pandas as pd

# Setup matplotlib backend
%matplotlib inline

# Setup seaborn
srs.set_theme()

In [11]:
# Load x-risk database estimates
df = pd.read_csv('./data/processed_estimates/all_estimates.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   estimator                      83 non-null     object 
 1   original_estimate              83 non-null     object 
 2   per_century_risk               83 non-null     float64
 3   estimate_confidence            83 non-null     object 
 4   conversion_confidence          83 non-null     object 
 5   reasoning                      83 non-null     object 
 6   risk_category                  83 non-null     object 
 7   estimate_confidence_numeric    83 non-null     int64  
 8   conversion_confidence_numeric  83 non-null     int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 6.0+ KB


## Motivation

Matters such as the extinction risk of humans are kind of a big deal, I find. And what these estimates do is reduce those complex matters to a number, a number that is comparable to the odds of your favorite team, or a horse winning a race. It might trivialize the odds, and it might give us a false sense of security.

These estimates are good for us to gauge our current risk of peril, but I find that looking at them in the context of humanity's potential future would be a good way to put things into perspective. It might give us a much-needed sense of urgency to reduce existential risk, and it might help us to prioritize.

So this is a little experiment to play around with the math, and see what comes out, maybe an interesting insight, and if not, at least it was fun.


## The Problem

Given that [humanity's potential to flourish is so incredibly vast](https://thoughtroamer.substack.com/p/the-extinction-game), what are our odds of fulfilling that potential? What are the odds that we peril before reaching that potential?

We want to embark in an exploration into existential risk. Particularly, we want to investigate how long we can expect humanity to still be around, given different existential risk levels. We can, of course, use current existential risk estimates, and see where that takes us, and we can also use varying levels, to see what would happen if we let them increase or if we get to work and successfully reduce them.

## The Extinction Game

Here, we will try to model the problem in different ways, starting from simple models to more complex ones.

Broadly the whole modeling is based on what I call "The Extinction Game" - yes, it's a dramatic subject. It's a game that involves the whole of humanity. In it, humanity *wins* and goes on for a long time full of flourishing and fulfilling its potential, or it *loses*, destroying its potential to flourish.

What exactly means to lose is debatable; it could either mean civilization's collapse, or extinction. However, the most generic scenario is one where *humanity's potential to flourish* is destroyed. This could be either due to full-fledged extinction, or it could be the establishment of an ultra stable dystopia that lasts for millions of years, or civilization's collapse without the ability to recover, but not necessarily the extinction of Homo Sapiens.

For brevity and clarity I'll use the term "extinction" to refer to the *losing* scenarios, even though it isn't necessarily synonymous with extinction.


### Level 1 - Constant Total Risk

This is the simplest model. In it, humanity's existential risk is modeled as a probability cascade, where the extinction game is played every century, and every century is independent - meaning that the outcome in one century doesn't affect the game in other centuries. So we use a constant figure for total risk.

To describe the problem we introduce the following events and expressions:

$$
\begin{align}
E&: \text{Extinction in a given century} \\ 
S&: \text{Survival in a given century} \\
P(E)&: \text{Probability of extinction in a given century} \rightarrow \text{Existential risk per century}\\
P(S)&: \text{Probability of survival in a given century} \\
X_n&: \text{Probability of extinction after } n \text{ periods (centuries)}
\end{align}
$$

Since the risk remains constant for each game, we are interested in the probabilities of still being around for, say 1000 years, or one million, or one billion.

We can describe the probability of being around for $n$ centuries by computing the odds that we don't get extinct for $n$ centuries in a row. We can assume that once the game is lost, it cannot be played again, so we're only interested in the scenario where survival is the outcome of every game.

$$
\begin{align}
P(X_n) = P(S)^n = [1-P(E)]^n
\end{align}
$$

In [ ]:
# Implement simple constant risk model
def probability_of_survival_after_n_centuries(n, xrisk):
    return (1 - xrisk) ** n

In [ ]:

# Plot the probability of survival after 10 centuries
n = np.arange(0, 100)
xrisk = 0.01

### Correlated Existential Risks

### Definitions

Let:
- $x_i(t)$ = risk from source $i$ in period $t$
- $X(t)$ = total existential risk in period $t$
- $\rho_{ij}$ = correlation coefficient between risks $i$ and $j$
- $n$ = number of risk sources (6 in our case)
- $\Sigma$ = correlation matrix with entries $\rho_{ij}$

Our risk sources are:
1. AI ($x_1$)
2. Nuclear ($x_2$)
3. Bio ($x_3$)
4. Natural ($x_4$)
5. Climate ($x_5$)
6. Dystopia ($x_6$)

## Base Risk Values (by 2100)
$x_1 = 0.10$ (AI)
$x_2 = 0.01$ (Nuclear)
$x_3 = 0.025$ (Bio)
$x_4 = 0.0001$ (Natural)
$x_5 = 0.001$ (Climate)
$x_6 = 0.05$ (Dystopia)

## Correlation Matrix
$\Sigma = \begin{pmatrix} 
1.0 & 0.3 & 0.4 & -0.3 & 0.2 & 0.7 \\
0.3 & 1.0 & 0.6 & 0.2 & 0.6 & 0.8 \\
0.4 & 0.6 & 1.0 & 0.4 & 0.5 & 0.7 \\
-0.3 & 0.2 & 0.4 & 1.0 & 0.8 & 0.5 \\
0.2 & 0.6 & 0.5 & 0.8 & 1.0 & 0.7 \\
0.7 & 0.8 & 0.7 & 0.5 & 0.7 & 1.0
\end{pmatrix}$

## Single Period Risk

For independent risks:
$X_{ind}(t) = 1 - \prod_{i=1}^n (1 - x_i(t))$

For correlated risks using a Gaussian copula:
$X(t) = 1 - C(1-x_1(t), 1-x_2(t), ..., 1-x_n(t); \Sigma)$

where $C$ is the Gaussian copula:
$C(u_1,...,u_n;\Sigma) = \Phi_\Sigma(\Phi^{-1}(u_1),...,\Phi^{-1}(u_n))$

Here:
- $\Phi_\Sigma$ is the multivariate normal CDF with correlation matrix $\Sigma$
- $\Phi^{-1}$ is the inverse of the standard normal CDF

## Multi-Period Survival

Probability of survival through $T$ periods:
$P(\text{survival through }T\text{ periods}) = \prod_{t=1}^T (1 - X(t))$

Expected survival time:
$E[T] = \sum_{t=1}^{\infty} t \cdot P(\text{survival exactly }t\text{ periods})$

where:
$P(\text{survival exactly }t\text{ periods}) = (1-X(t))^{t-1} \cdot X(t)$